# Part I: Fundamentals of NLP

## **Topic Modelling**

Natural Language Processing  

Author: D.Thébault

References used:  
- https://nlpdemystified.org

- https://www.khoury.northeastern.edu/home/vip/teach/DMcourse/5_topicmodel_summ/Lad-Gibbs-lecture032321.pdf

- https://arxiv.org/pdf/1505.02065v4

- https://www.youtube.com/watch?v=4DsgCZ3KTNk&t=1336s

**In this notebook we will code the Collapsed Gibbs Sampling for LDA model.**  
We will start by an introduction about CGS for LDA.  
Then we will import the packages, load the corpus and preprocess it.  
Finally, we will code the CGS for LDA step by step.

### Colapsed Gibbs Sampling (CGS) for LDA

**How do we go from data to a set of topics describing that data ?**

Inferring these distributions directly is intractable (prohibitive amount of computational resources).  
Instead we use an approximate inference technique called <u>**Collapsed Gibbs Sampling (CGS)**</u>.  

CGS simplifies the inference because we don't have to sample explicitly the topics and words distributions.  

Instead, CGS estimates these distributions by sampling the topics associated with the words in the corpus. 

**Monte Carlo Markov Chain (MCMC)**  

MCMC is a classical algorithm for sampling from multivariate probability distributions.  
It provides an easy way to sample from high dimensional distributions.  

For a joint distibution that is difficult to sample from directly, Gibbs sampling  
is a way to sample only one variable at a time while fixing the values of the others variables.  
In the CGS, the conditional probability which is maximized for topic modelling (LDA)  
is the probability to assign a topic $z$ to a word $w$ in a document $d$.  
For each word $w$ in a document $d$, the algorithm maximizes the conditional probability:  

$p(z \mid w, d, z_{-i})$  

where $z_{-i}$ represents the assignments of topics for all words except $i$.  
The word $i$ is temporarly deleted of counting to avoid a bias in the sample.  


### The CGS algorithm  

- Part 1. Initialization of the matrices,  

- Part 2. Update the topic assignments for each word in each document.  

    Loop:  
    For each word in each document, we will:  
        - 1. Remove the current topic assignment.  
        - 2. Compute the probability of assigning the word to each topic.  
        - 3. Sample a new topic assignment based on the computed probabilities.  
        - 4. Update the count matrices with the new topic assignment.  

The probability to assign a word $w_n$ to a topic $k$ in a document $d$ is given by:  

$$p(z_{dn}=k \mid w, \alpha, \beta) 
\propto 
\dfrac{n_{dk}^{-dn} + \alpha_k}{\underset{k'}{\sum}(n_{dk'}^{-dn} + \alpha_{k'})} 
\times
\dfrac{n_{kw_{dn}}^{-dn} + \beta_{w_{dn}}}{\underset{w'}{\sum}(n_{kw'}^{-dn} + \beta_{w'})}$$

This formula combines the probability of assigning a topic $k$ to a document $d$  
and the probability of assigning a word $w+{dn}$ to a topic $k$.  

### STEP 0: Preprocessing

#### The packages

In [52]:
import numpy as np
import pandas as pd
from collections import Counter
from collections import defaultdict
import random
import spacy

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [53]:
# Options
# configuration of the numpy arrays to avoid scientific outputs
np.set_printoptions(suppress=True, precision=4)

# reproductibility
np.random.seed(42)
random.seed(42)

#### Load the corpus

In [54]:
corpus = fetch_20newsgroups(categories=['sci.space'],remove=('headers', 'footers', 'quotes'))

# Select the first documents
D = 3
first_documents = corpus.data[:D]

# Print the first documents
for doc in first_documents:
    print(doc)
    print("\n" + "-"*80 + "\n")  # Separator for better readability

# Split the text into words and count the number of words
words = first_documents[0].split()
print(f"\n Number of words of the first document: {len(words)}")


Any lunar satellite needs fuel to do regular orbit corrections, and when
its fuel runs out it will crash within months.  The orbits of the Apollo
motherships changed noticeably during lunar missions lasting only a few
days.  It is *possible* that there are stable orbits here and there --
the Moon's gravitational field is poorly mapped -- but we know of none.

Perturbations from Sun and Earth are relatively minor issues at low
altitudes.  The big problem is that the Moon's own gravitational field
is quite lumpy due to the irregular distribution of mass within the Moon.

--------------------------------------------------------------------------------


Glad to see Griffin is spending his time on engineering rather than on
ritual purification of the language.  Pity he got stuck with the turkey
rather than one of the sensible options.

--------------------------------------------------------------------------------




In spite of my great respect for the people you speak of, I think thei

#### Preprocessing

##### corpus -> processed_corpus

In [55]:
%%time
# Load the SpaCy english language model "en_core_web_sm"
# This model inclused functionalities such as tokenisation, 
# POS tagging (étiquetage des parties du discours), la reconnaissance des entités nommées (NER), 
# and syntactic analysis (parsing).
nlp = spacy.load('en_core_web_sm')

# We don't need named-entity recognition nor dependency parsing in the pipeline. 
# We do need part-of-speech tagging however.
unwanted_pipes = ["ner", "parser"]

# Preprocessing
# For this exercise, we'll remove punctuation, spaces (which
# includes newlines) and stopwords filter for tokens consisting of alphabetic
# characters, and return the lemma (which require POS tagging).
def spacy_tokenizer(doc):
  with nlp.disable_pipes(*unwanted_pipes):
    return [t.lemma_ for t in nlp(doc) if \
            not t.is_punct and \
            not t.is_space and \
            not t.is_stop and \
            t.is_alpha]  # Check if the token is not a named entity (we want to remove Names)

# Apply the preprocessing function to the corpus
processed_corpus = []
for text in first_documents:
    processed_text = spacy_tokenizer(text)
    processed_corpus.append(processed_text)

# Print the first document in the processed corpus to verify
for doc in processed_corpus:
    print(doc)
    print("\n" + "-"*80 + "\n")  # Separator for better readability

['lunar', 'satellite', 'need', 'fuel', 'regular', 'orbit', 'correction', 'fuel', 'run', 'crash', 'month', 'orbit', 'Apollo', 'mothership', 'change', 'noticeably', 'lunar', 'mission', 'last', 'day', 'possible', 'stable', 'orbit', 'Moon', 'gravitational', 'field', 'poorly', 'map', 'know', 'perturbation', 'Sun', 'Earth', 'relatively', 'minor', 'issue', 'low', 'altitude', 'big', 'problem', 'Moon', 'gravitational', 'field', 'lumpy', 'irregular', 'distribution', 'mass', 'Moon']

--------------------------------------------------------------------------------

['Glad', 'Griffin', 'spend', 'time', 'engineer', 'ritual', 'purification', 'language', 'Pity', 'get', 'stick', 'turkey', 'sensible', 'option']

--------------------------------------------------------------------------------

['spite', 'great', 'respect', 'people', 'speak', 'think', 'cost', 'estimate', 'bit', 'optimistic', 'work', 'SSTO', 'complex', 'large', 'airliner', 'small', 'experience', 'base', 'SSTO', 'development', 'cost', 'typi

#### Initialization of the parameters

We have to initialize the LDA parameters.  
The matrix of counting the words and the topics.  

##### Parameters of the LDA  

- $K$: Number of topics  

- $\alpha$: Parameter of the Dirichlet distribution for the topics of the documents

- $\beta$: Parameter of the Dirichlet distribution for the words in the subjects.  

- NUM_ITER: Number of iterations. 

In [56]:
np.random.seed(42)

# Exemple d'utilisation
K = 3
ALPHA = 0.5
BETA = 0.5
NUM_ITER = 20

num_docs = len(processed_corpus)
topic_list = list(range(K))

print("NUM_TOPICS = ", K)
print("num_docs = ", num_docs)
print("topic list = ", topic_list)

NUM_TOPICS =  3
num_docs =  3
topic list =  [0, 1, 2]


### STEP 1: Random assignation

#### Randomly assign a topic to every word

In [57]:
np.random.seed(42)

# STEP 1: Randomly assign a topic number to every word in every document.
# ======
z = []
for _, doc in enumerate(processed_corpus):
    doc_with_topics = [(word, np.random.randint(low=0, high=K)) for word in doc]
    z.append(doc_with_topics)

z

[[('lunar', 2),
  ('satellite', 0),
  ('need', 2),
  ('fuel', 2),
  ('regular', 0),
  ('orbit', 0),
  ('correction', 2),
  ('fuel', 1),
  ('run', 2),
  ('crash', 2),
  ('month', 2),
  ('orbit', 2),
  ('Apollo', 0),
  ('mothership', 2),
  ('change', 1),
  ('noticeably', 0),
  ('lunar', 1),
  ('mission', 1),
  ('last', 1),
  ('day', 1),
  ('possible', 0),
  ('stable', 0),
  ('orbit', 1),
  ('Moon', 1),
  ('gravitational', 0),
  ('field', 0),
  ('poorly', 0),
  ('map', 2),
  ('know', 2),
  ('perturbation', 2),
  ('Sun', 1),
  ('Earth', 2),
  ('relatively', 1),
  ('minor', 1),
  ('issue', 2),
  ('low', 1),
  ('altitude', 2),
  ('big', 2),
  ('problem', 0),
  ('Moon', 2),
  ('gravitational', 0),
  ('field', 2),
  ('lumpy', 2),
  ('irregular', 0),
  ('distribution', 0),
  ('mass', 2),
  ('Moon', 1)],
 [('Glad', 0),
  ('Griffin', 1),
  ('spend', 1),
  ('time', 1),
  ('engineer', 0),
  ('ritual', 1),
  ('purification', 0),
  ('language', 1),
  ('Pity', 2),
  ('get', 2),
  ('stick', 0),
  ('tur

### STEP 2: Counting


- $n_{dk}$ : number of words in the document $d$ assigned to the topic $k$ $\iff$ number of times topic $k$ appears in a document $d$   

- $n_{w,k}$ : number of times where the word $w$ is assigned to the topic $k$  

- $n_k$ : number of words assigned to the topic $k$  

- $n_d$ : number of words in the document $d \; :$ $\sum_{i=1}^K{n_{d,i}}$  

- $n$ : number of words in the corpus.

In [58]:
# STEP 2:

# Number of words per topic per document
ndk = []
for _, doc_with_topic in enumerate(z):
    topic_counts = defaultdict(int)
    for word, topic in doc_with_topic:
        topic_counts[topic] += 1
    ndk.append(topic_counts)

# Number of times a word is assigned to a topic k across the corpus.
nwk = defaultdict(lambda: defaultdict(int))
for _, doc_with_topic in enumerate(z):
    for word, topic in doc_with_topic:
        nwk[topic][word] += 1
nwk = dict(nwk)

# Number of words per topics across the corpus
nk = defaultdict(int)
for dico in ndk:
    for key, value in dico.items():
        nk[key] += value

# Number of words per document
nd = defaultdict(int)
for i, dico in enumerate(ndk):
    nd[i] = sum(dico.values())

# Number of words of the corpus:
n=0
for doc in processed_corpus:
    n+=len(doc)

ndk, nwk, nk, nd, n

([defaultdict(int, {2: 20, 0: 14, 1: 13}),
  defaultdict(int, {0: 4, 1: 6, 2: 4}),
  defaultdict(int, {0: 18, 1: 18, 2: 13})],
 {2: defaultdict(int,
              {'lunar': 1,
               'need': 1,
               'fuel': 1,
               'correction': 1,
               'run': 1,
               'crash': 1,
               'month': 1,
               'orbit': 1,
               'mothership': 1,
               'map': 1,
               'know': 1,
               'perturbation': 1,
               'Earth': 1,
               'issue': 1,
               'altitude': 1,
               'big': 1,
               'Moon': 1,
               'field': 1,
               'lumpy': 1,
               'mass': 1,
               'Pity': 1,
               'get': 1,
               'turkey': 1,
               'sensible': 1,
               'optimistic': 1,
               'experience': 1,
               'base': 1,
               'development': 1,
               'g': 1,
               'market': 2,
               'giv

### STEP 3: Unassign / Reassign topic

##### In a document d, unassign a word from its topic  

#### Assign w a new topic based on:  
- a. How much this document d likes topic k  
- b. How much this topic likes word w

In [62]:
# STEP 3: Unassign topic, reassign topic

count_display = 0 # to control the displaying of the following probability
for _ in range(NUM_ITER):
    for doc_idx, doc in enumerate(processed_corpus):
        # print("doc_idx: ", doc_idx)
        for i in range(len(doc)):
            word = doc[i]
            # print("word: ", word)
            # print("z[doc_idx][i][1]: ", z[doc_idx][i][1])
            topic = z[doc_idx][i][1]
            # print("topic: ", topic)

            # In a document d, unassign a word from its topic
            ndk[doc_idx][topic] -= 1
            if ndk[doc_idx][topic] < 0:
                del ndk[doc_idx][topic]
            nwk[topic][word] -= 1
            if nwk[topic][word] < 0:
                del nwk[topic][word]
            nk[topic] -= 1

            # Assign w a new topic based on:
            # a) The prevalence of each topic in the document:
            #    Nb of times topic occurs in doc_idx + ALPHA / Total number of words in the doc_idx + K * ALPHA
            # p_theta_d = (ndk[doc_idx][topic] + ALPHA) / (nd[doc_idx] + K * ALPHA)

            # b) The prevalence of the word in each topic:
            #    Nb of times word occurs in topic across corpus + BETA / Nb of words assigned to topic across corpus + n * BETA
            # p_phi_k = (nwk[topic][word] + BETA) / (nk[topic] + n * BETA)

            # Version 1:
            p_z = [(ndk[doc_idx][k] + ALPHA) / (nd[doc_idx] + K * ALPHA) * \
                   (nwk[topic][word] + BETA) / (nk[k] + n * BETA) \
                   for k in range(K)]
            
            # Version 2:
            p_z = [(ndk[doc_idx][k] + ALPHA) * (nwk[topic][word] + BETA) / (nk[k] + n * BETA) \
                   for k in range(K)]
            
            # Display the set of probabilities
            if count_display < 5:
                print(p_z)
                count_display+=1

            topic = random.choices(topic_list, weights=p_z, k=1)[0]
            # topic = p_z.index(max(p_z)) # with the max
            
            # update n parameters
            z[doc_idx][i] = word, topic
            ndk[doc_idx][topic] += 1
            nwk[topic][word] += 1
            nk[topic] += 1

[0.31451612903225806, 0.007575757575757576, 0.5030487804878049]
[0.10483870967741936, 0.0025252525252525255, 0.1676829268292683]
[0.10483870967741936, 0.0025252525252525255, 0.1676829268292683]
[0.3271276595744681, 0.007575757575757576, 0.49074074074074076]
[0.11315789473684211, 0.0025252525252525255, 0.159375]


### ALL IN ONE

In [67]:
import random
from collections import defaultdict
np.random.seed(42)

def lda_collapsed_gibs(corpus, num_iter=20, K=3, ALPHA=0.1, BETA=0.1):

    # STEP 1: Randomly assign a topic number to every word in every document.

    z = []
    for _, doc in enumerate(corpus):
        doc_with_topics = [(word, np.random.randint(low=0, high=K)) for word in doc]
        z.append(doc_with_topics)


    # STEP 2: Counting

    # Number of words per topic per document
    ndk = []
    for _, doc_with_topic in enumerate(z):
        topic_counts = defaultdict(int)
        for word, topic in doc_with_topic:
            topic_counts[topic] += 1
        ndk.append(topic_counts)

    # Number of times a word is assigned to a topic k across the corpus.
    nwk = defaultdict(lambda: defaultdict(int))
    for _, doc_with_topic in enumerate(z):
        for word, topic in doc_with_topic:
            nwk[topic][word] += 1
    nwk = dict(nwk)

    # Number of words per topics across the corpus
    nk = defaultdict(int)
    for dico in ndk:
        for key, value in dico.items():
            nk[key] += value

    # Number of words per document
    nd = defaultdict(int)
    for i, dico in enumerate(ndk):
        nd[i] = sum(dico.values())

    # Number of words of the corpus:
    n=0
    for doc in corpus:
        n+=len(doc)


    # STEP 3. Unassign, reassign topics

    for _ in range(NUM_ITER):
        for doc_idx, doc in enumerate(corpus):
            for i in range(len(doc)):
                word = doc[i]
                topic = z[doc_idx][i][1]

                # In a document d, unassign a word from its topic
                ndk[doc_idx][topic] -= 1
                if ndk[doc_idx][topic] < 0:
                    del ndk[doc_idx][topic]
                nwk[topic][word] -= 1
                if nwk[topic][word] < 0:
                    del nwk[topic][word]
                nk[topic] -= 1

                # Assign w a new topic

                p_z = [(ndk[doc_idx][k] + ALPHA) * (nwk[topic][word] + BETA) / (nk[k] + n * BETA) \
                    for k in range(K)]
                
                topic = random.choices(topic_list, weights=p_z, k=1)[0]
                # topic = p_z.index(max(p_z)) # with the max
                
                # update n parameters
                z[doc_idx][i] = word, topic
                ndk[doc_idx][topic] += 1
                nwk[topic][word] += 1
                nk[topic] += 1

    # STEP 4: Count the most frequent words per document
    word_counts_per_doc = []
    for doc_with_topics in z:
        word_counts = Counter(word for word, _ in doc_with_topics)
        word_counts_per_doc.append(word_counts)

    return z, ndk, nwk, nk, word_counts_per_doc

In [68]:
z, ndk, nwk, nk, word_counts_per_doc = lda_collapsed_gibs(processed_corpus,ALPHA=20)

# Print the most frequent words per document
for doc_idx, word_counts in enumerate(word_counts_per_doc):
    print(f"Document {doc_idx}: {word_counts.most_common(5)}")

Document 0: [('orbit', 3), ('Moon', 3), ('lunar', 2), ('fuel', 2), ('gravitational', 2)]
Document 1: [('Glad', 1), ('Griffin', 1), ('spend', 1), ('time', 1), ('engineer', 1)]
Document 2: [('think', 3), ('cost', 3), ('SSTO', 3), ('large', 2), ('airliner', 2)]


# APPENDICES

### Numpy alternative

[Youtube - Alladin Persson](https://www.youtube.com/watch?v=aPRjj8i_6yE&t=161s)